In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import snowflake.connector

### Loading the Data

In [9]:
df_raw = pd.read_csv("./data/train_dataset.csv")

### Splitting the Data

In [11]:
train_data,test_data = train_test_split(df_raw,test_size=0.2,train_size=0.8)

### Training the model

In [14]:
model = RandomForestClassifier(n_estimators=100)
model.fit(train_data.iloc[:,:-1],train_data['smoking'])

RandomForestClassifier()

In [15]:
model.score(test_data.iloc[:,:-1],test_data['smoking'])

0.8097986405027575

In [16]:
df_test = pd.read_csv('./data/test_dataset.csv')

In [20]:
df_raw.write

TypeError: to_sql() missing 2 required positional arguments: 'name' and 'con'

In [21]:
print()

SIVACHANDAN


In [3]:
with snowflake.connector.Connect(user = os.getenv('SNOWSQL_USER'),password = os.getenv('SNOWSQL_PASSWORD'),account = os.getenv('SNOWSQL_ACCOUNT')) as conn:
    cs = conn.cursor()
    cs.execute("SELECT current_version()")
    one_row = cs.fetchone()
    print(one_row[0])
    

7.3.1
